# Frozen Lake

In [1]:
#Modified frozen lake example, see: https://gym.openai.com/envs/FrozenLake-v0/

using Random

### Auxilliary Functions

In [62]:
# actions coded as :left => 1, :down => 2, :right => 3, :up => 4
actions = Dict(1 => [0,-1], 2 => [1,0], 3 => [0,1], 4 => [-1,0]);

#arrows are corresponding to actions
arrows = Dict(1 => '⇐', 2 => '⇓', 3 => '⇒', 4 => '⇑');

rewards = Dict('S' => -0.05, 'G' => 1.0, 'H' => -1.0, 'F' => -0.05);

grid4x4= ['S' 'F' 'F' 'F';
        'F' 'H' 'F' 'H';
        'F' 'F' 'F' 'H';
        'H' 'F' 'F' 'G'];

grid8x8 =['S' 'F' 'F' 'F' 'F' 'F' 'F' 'F';
        'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F';
        'F' 'F' 'F' 'H' 'F' 'F' 'F' 'F';
        'F' 'F' 'F' 'F' 'F' 'H' 'F' 'F';
        'F' 'F' 'F' 'H' 'F' 'F' 'F' 'F';
        'F' 'H' 'H' 'F' 'F' 'F' 'H' 'F';
        'F' 'H' 'F' 'F' 'H' 'F' 'H' 'F';
        'F' 'F' 'F' 'H' 'F' 'F' 'F' 'G';];

function get_grid(dim, p_holes, seed = 234)
    Random.seed!(seed)
    grid = [rand() < p_holes ? 'H' : 'F' for i in 1:dim, j in 1:dim]
    grid[1,1] = 'S'
    grid[end,end] = 'G'
    return grid
end



#Transition matrix T(s',a,s) - from state s to successor state s' by taking action a 
#assuming that ice is slippery and probability of moving forward is equal to 0.8, 
#probabilities of moving left or right are equal to 0.1

function transition_matrix(grid, actions = actions)
    T = zeros(length(grid),length(actions),length(grid))
    i2s = CartesianIndices(grid)
    s2i = LinearIndices(grid)
    for i = 1:length(grid)
        if !(grid[i] == 'H' || grid[i] == 'G')
            index = i2s[i]
            for j = 1:length(actions)
                indices = Tuple(index) .+ actions[j]
                if all(in.( indices, (1:size(grid,1), 1:size(grid,2)))) 
                    k = s2i[indices...]
                    T[k,j,i] += 0.8
                    if actions[j][1] == 0
                        for l in [-1,1]
                            ind = Tuple(index) .+ (l,0)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2)))) 
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    else
                        for l in [-1,1]
                            ind = Tuple(index) .+ (0,l)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2))))
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    end
                else
                    T[i,j,i] += 0.8
                    if actions[j][1] == 0
                        for l in [-1,1]
                            ind = Tuple(index) .+ (l,0)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2)))) 
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    else
                        for l in [-1,1]
                            ind = Tuple(index) .+ (0,l)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2)))) 
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    end
                end
            end
        end
    end
    return T
end

function reward_matrix(grid, rewards = rewards)
    R = zeros(size(grid))
    for i = 1:length(grid)
        R[i] = rewards[grid[i]]
    end
    return R
end


random_policy(grid,actions = actions) = rand(1:length(actions),size(grid))


function print_policy(P, grid, arrows = arrows)
    Policy = rand(Char,size(grid))
    for i = 1:length(grid)
        if grid[i] == 'F' || grid[i] == 'S' 
            Policy[i] = arrows[P[i]]
        elseif grid[i] == 'H' 
            Policy[i] = '⦷'
        else
            Policy[i] = grid[i]
        end
    end
    return Policy
end

print_policy (generic function with 2 methods)

### Policy Evaluation Functions

In [63]:
#synchronous DP
function evaluate!(P, v, v₁, R, T, β)
    for s = 1:length(v)
        v₁[s]= R[s] + β * sum(v .*  T[:,P[s],s])
    end
end 

#in-place DP
function evaluate!(P, v, R, T, β)
    for s = 1:length(v)
        v[s]= R[s] + β * sum(v .*  T[:,P[s],s])
    end
end 

evaluate! (generic function with 2 methods)

### Policy Evaluation


In [64]:
function evaluate_policy(grid,P;
                        β = 0.999, ϵ=0.0001, 
                        actions = actions)
    iter = 0
    T = transition_matrix(grid)
    R = reward_matrix(grid)
    v₁ = zeros(length(grid))
    while true
        iter += 1
        v = deepcopy(v₁)
        evaluate!(P, v, v₁, R, T, β)
        #@info v₁
        δ = maximum(abs.(v₁ - v)) 
        δ < ϵ * (1 - β) / β && break 
    end 
    
    println("Iterations: $(iter)")
    return reshape(v₁,size(grid)),  print_policy(P, grid)
end

evaluate_policy (generic function with 1 method)

In [65]:
v, p = evaluate_policy(grid4x4, random_policy(grid4x4))

Iterations: 2148


([-5.14908660773023 -7.985478441301969 -8.806092629243011 -9.214356335655268; -4.738417925735517 -1.0 -1.913123221414557 -1.0; -4.494073679299217 -2.083805322378003 -1.8437895755390197 -1.0; -1.0 -1.7525685668096453 0.6277140572427211 1.0], ['⇓' '⇒' '⇑' '⇑'; '⇑' '⦷' '⇒' '⦷'; '⇑' '⇓' '⇐' '⦷'; '⦷' '⇑' '⇒' 'G'])

In [66]:
v

4×4 Matrix{Float64}:
 -5.14909  -7.98548  -8.80609   -9.21436
 -4.73842  -1.0      -1.91312   -1.0
 -4.49407  -2.08381  -1.84379   -1.0
 -1.0      -1.75257   0.627714   1.0

In [67]:
p

4×4 Matrix{Char}:
 '⇓'  '⇒'  '⇑'  '⇑'
 '⇑'  '⦷'  '⇒'  '⦷'
 '⇑'  '⇓'  '⇐'  '⦷'
 '⦷'  '⇑'  '⇒'  'G'

### Value Iteration Algorithm

In [76]:

#value iteration algorithm

function get_policy(v, T, actions = actions)
    P = rand(Int,length(v))
    for s = 1:length(v)
        actions_vector = zeros(length(actions))
        for i = 1:length(actions)
            actions_vector[i] = sum(v .* T[:,i,s])
        end
        P[s] = argmax(actions_vector)
    end 
    return P
end

function update_values!(v₁, v, T,R,β, actions = actions)
    for s = 1:length(v₁)
        actions_vector = zeros(length(actions))
        for a = 1:length(actions)
            actions_vector[a] = sum(v .* T[:,a,s])
        end
        v₁[s] = R[s] +  β * maximum(actions_vector)
    end
end

 function value_iteration(grid,β = 0.999, ϵ=0.0001, actions = actions)
    iter = 0
    T = transition_matrix(grid)
    R = reward_matrix(grid)
    v₁ = zeros(length(grid))
    while true
        iter += 1
        v = deepcopy(v₁)
        update_values!(v₁,v, T,R,β)
        δ = maximum( abs.(v₁ - v))  
        δ < ϵ * (1 - β) / β && break
    end
    P = get_policy(v₁, T)
    println("Iterations: $(iter)")
    return reshape(v₁,(size(grid))), print_policy(P, grid)
end


vᵥ, pᵥ =  value_iteration(grid8x8);

Iterations: 55


In [77]:
vᵥ

8×8 Matrix{Float64}:
 -0.339438  -0.27149   -0.202868  …  -0.00621787   0.0505643  0.102791
 -0.385921  -0.322684  -0.25832       0.0399633    0.113759   0.172011
 -0.441663  -0.391518  -0.395462     -0.00332438   0.178408   0.24207
 -0.498583  -0.45843   -0.457666     -1.0          0.236042   0.312893
 -0.556933  -0.528808  -0.568494     -0.00192918   0.181735   0.385454
 -0.661038  -1.0       -1.0       …   0.0273081   -1.0        0.473963
 -0.753474  -1.0       -0.579584      0.238882    -1.0        0.721364
 -0.811142  -0.780109  -0.691036      0.611464     0.721364   1.0

In [78]:
pᵥ

8×8 Matrix{Char}:
 '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇓'  '⇓'
 '⇒'  '⇒'  '⇒'  '⇑'  '⇒'  '⇒'  '⇓'  '⇓'
 '⇑'  '⇑'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇑'  '⇒'  '⇑'  '⦷'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇑'  '⦷'  '⦷'  '⇒'  '⇒'  '⇓'  '⦷'  '⇓'
 '⇑'  '⦷'  '⇒'  '⇑'  '⦷'  '⇓'  '⦷'  '⇓'
 '⇑'  '⇒'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  'G'

### Policy Iteration Algorithm


In [119]:
function evaluate!(π, v, v₁, R, T, β)
    for s = 1:length(v)
        v₁[s]= R[s] + β * sum(v .*  T[:,π[s],s])
    end
end 

function evaluate_policy!(v₁, T, R, π;
                        β = 0.999, ϵ=0.0001,
                        actions = actions)
    while true        
        v = deepcopy(v₁)
        evaluate!(π, v, v₁, R, T, β)
        δ = maximum(abs.(v₁ - v)) 
        δ < ϵ * (1 - β) / β && break 
    end 
end


function improve_policy!(v, T, P, actions = actions)
    done = true
    for s = 1:length(v)
        actions_vector = zeros(length(actions))
        for a = 1:length(actions)
            actions_vector[a] = sum(v .* T[:,a,s])
        end
        action = argmax(actions_vector)
        action != P[s] && (P[s] = action; done = false)
    end
    return done
end

function policy_iteration(grid,β = 0.999, ϵ=0.0001)
    iter = 1
    T = transition_matrix(grid)
    R = reward_matrix(grid)
    v₁ = zeros(length(grid))
    P = random_policy(grid)
    done = false
    while !done 
        iter += 1
        done = true 
        evaluate_policy!(v₁, T, R, P,
                        β = β, ϵ = ϵ,
                        actions = actions)
        done = improve_policy!(v₁, T, P)
    end 
    println("Iterations: $(iter)")
    return reshape(v₁,size(grid)),  print_policy(P, grid)
end

 vₚ, pₚ = policy_iteration(grid8x8);


Iterations: 10


In [120]:
maximum(abs.(vₚ .- vᵥ))

1.3110764818247134e-7

In [122]:
println(sum(pᵥ .== pₚ) / length(pᵥ))

1.0


In [123]:
pₚ

8×8 Matrix{Char}:
 '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇓'  '⇓'
 '⇒'  '⇒'  '⇒'  '⇑'  '⇒'  '⇒'  '⇓'  '⇓'
 '⇑'  '⇑'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇑'  '⇒'  '⇑'  '⦷'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇑'  '⦷'  '⦷'  '⇒'  '⇒'  '⇓'  '⦷'  '⇓'
 '⇑'  '⦷'  '⇒'  '⇑'  '⦷'  '⇓'  '⦷'  '⇓'
 '⇑'  '⇒'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  'G'

### Experiments - how rewards impact the optimal solution

In [29]:
πs = []
for α in [-0.2,-0.05,0.0,0.05,.1]
    rewards = Dict('S' => α, 'G' => 1.0, 'H' => -1.0, 'F' => α);
    vₚ, pₚ = policy_iteration(grid8x8);
    vᵥ, pᵥ = value_iteration(grid8x8);
    println(pₚ == pᵥ)
    push!(πs,pₚ)
end

Iterations: 8
Iterations: 35
true
Iterations: 11
Iterations: 55
true
Iterations: 10
Iterations: 481
true
Iterations: 17
Iterations: 13106
false
Iterations: 21
Iterations: 13805
true


In [19]:
πs[1]

8×8 Matrix{Char}:
 '⇒'  '⇒'  '⇓'  '⇓'  '⇓'  '⇓'  '⇓'  '⇓'
 '⇒'  '⇒'  '⇓'  '⇓'  '⇓'  '⇓'  '⇓'  '⇓'
 '⇒'  '⇒'  '⇒'  '⦷'  '⇐'  '⇒'  '⇓'  '⇓'
 '⇓'  '⇓'  '⇓'  '⇓'  '⇓'  '⦷'  '⇒'  '⇓'
 '⇓'  '⇓'  '⇓'  '⦷'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇒'  '⦷'  '⦷'  '⇒'  '⇒'  '⇓'  '⦷'  '⇓'
 '⇒'  '⦷'  '⇑'  '⇒'  '⦷'  '⇓'  '⦷'  '⇓'
 '⇒'  '⇑'  '⇒'  '⦷'  '⇒'  '⇒'  '⇒'  'G'

In [20]:
πs[2]

8×8 Matrix{Char}:
 '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇓'  '⇓'
 '⇒'  '⇒'  '⇒'  '⇑'  '⇒'  '⇒'  '⇓'  '⇓'
 '⇑'  '⇑'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇑'  '⇒'  '⇑'  '⦷'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇑'  '⦷'  '⦷'  '⇒'  '⇒'  '⇓'  '⦷'  '⇓'
 '⇑'  '⦷'  '⇒'  '⇑'  '⦷'  '⇓'  '⦷'  '⇓'
 '⇑'  '⇒'  '⇑'  '⦷'  '⇒'  '⇒'  '⇒'  'G'

In [21]:
πs[3]

8×8 Matrix{Char}:
 '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇒'  '⇓'
 '⇒'  '⇒'  '⇑'  '⇑'  '⇑'  '⇒'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇐'  '⦷'  '⇒'  '⇑'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇐'  '⇐'  '⇑'  '⦷'  '⇒'  '⇓'
 '⇑'  '⇑'  '⇑'  '⦷'  '⇒'  '⇒'  '⇑'  '⇓'
 '⇐'  '⦷'  '⦷'  '⇒'  '⇑'  '⇐'  '⦷'  '⇒'
 '⇐'  '⦷'  '⇓'  '⇐'  '⦷'  '⇓'  '⦷'  '⇒'
 '⇑'  '⇓'  '⇐'  '⦷'  '⇒'  '⇒'  '⇓'  'G'

In [22]:
πs[4]

8×8 Matrix{Char}:
 '⇐'  '⇐'  '⇐'  '⇐'  '⇐'  '⇐'  '⇐'  '⇐'
 '⇑'  '⇑'  '⇑'  '⇑'  '⇑'  '⇑'  '⇑'  '⇑'
 '⇐'  '⇐'  '⇐'  '⦷'  '⇒'  '⇑'  '⇑'  '⇑'
 '⇐'  '⇐'  '⇐'  '⇐'  '⇑'  '⦷'  '⇒'  '⇑'
 '⇐'  '⇑'  '⇐'  '⦷'  '⇒'  '⇒'  '⇑'  '⇑'
 '⇐'  '⦷'  '⦷'  '⇒'  '⇑'  '⇐'  '⦷'  '⇒'
 '⇐'  '⦷'  '⇓'  '⇐'  '⦷'  '⇑'  '⦷'  '⇑'
 '⇐'  '⇓'  '⇐'  '⦷'  '⇒'  '⇑'  '⇐'  'G'

In [31]:
πs[end]

8×8 Matrix{Char}:
 '⇓'  '⇐'  '⇐'  '⇐'  '⇐'  '⇐'  '⇐'  '⇐'
 '⇑'  '⇑'  '⇑'  '⇑'  '⇑'  '⇑'  '⇑'  '⇑'
 '⇐'  '⇐'  '⇐'  '⦷'  '⇒'  '⇑'  '⇑'  '⇑'
 '⇐'  '⇐'  '⇐'  '⇐'  '⇑'  '⦷'  '⇒'  '⇑'
 '⇐'  '⇑'  '⇐'  '⦷'  '⇒'  '⇒'  '⇑'  '⇑'
 '⇐'  '⦷'  '⦷'  '⇒'  '⇑'  '⇐'  '⦷'  '⇒'
 '⇐'  '⦷'  '⇓'  '⇐'  '⦷'  '⇑'  '⦷'  '⇑'
 '⇐'  '⇓'  '⇐'  '⦷'  '⇒'  '⇑'  '⇐'  'G'